### Матричные факторизации


Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© д ©«.


В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [2]:

import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

C:\Users\ASUS\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')
ratings

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [4]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)',
 '1526    Hercules (1997)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [17]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '847    Godfather, The (1972)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1892    Rain Man (1988)',
 '453    Fugitive, The (1993)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

Сделаем сначала с рейтингом тоже самое что и с implicit данными (для rmse)

In [18]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item_exp = sp.coo_matrix((ratings["rating"], (users, movies)))

Задание состоит в разделении матрицы на две - юзеры и фильмы, чтобы рекомендовать и находить похожие, мне удобно будет сделать класс, от которого 4 раза можно отнаследоваться

Очевидно нам понадобится rmse и 2 предсказателя

In [19]:
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
import math
class Base:
    def __init__(self, matrix, ratings, k, amount_of_users, amount_of_movies, start):
        self.ratings = ratings
        self.matrix = matrix
        self.k = k
        self.U = np.random.uniform(0, start / math.sqrt(k), (amount_of_users, k))
        self.M = np.random.uniform(0, start / math.sqrt(k), (k, amount_of_movies))
        self.B_u = np.zeros((amount_of_users, 1))
        self.B_m = np.zeros((1, amount_of_movies))
        self.mu = ratings.rating.mean() 
    
    def rmse(self):
        ans = self.ratings
        matr2 = (self.U @ self.M)[ans["user_id"], ans["movie_id"]]
        return metrics.mean_squared_error(self.ratings["rating"], matr2, squared=True)
    def make_recomendation(self, u_id):
        print('User films')
        print()
        u_films = get_user_history(u_id, ratings)[:10]
        print(*u_films, sep='\n')
        print()
        print('Recomendation')
        print()
        prediction = self.U @ self.M
        rating = np.argsort(prediction[u_id])[::-1]
        return rating
        
    
    def make_recomendation(self, u_id):
        print('User films')
        print()
        u_films = get_user_history(u_id, self.ratings)[:10]
        print(*u_films, sep='\n')
        print()
        print('Recomendation')
        print()
        prediction = self.U @ self.M
        rating = np.argsort(prediction[u_id])[::-1]
        for i in rating[:10]:
            print(*movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
        return

    def find_similar1(self, m_id):
        print('Film', movie_info[movie_info['movie_id'] == m_id][['name', 'category']].to_string())
        similarity = np.dot(self.M[:,m_id], self.M) / np.sqrt((self.M * self.M).sum(axis=0)) / np.sqrt((self.M[:,m_id] ** 2).sum())
        recomend = np.argsort(similarity)[::-1]
        for i in recomend[:10]:
            print(movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
        return
        
    def find_similar2(self, m_id):
        movies = self.M.T
        print('Film', movie_info[movie_info['movie_id'] == m_id][['name', 'category']].to_string())
        matr1 = movies[m_id]
        matr2 = movies
        similarity = cosine_similarity(np.expand_dims(matr1, axis=0), self.M.T)[0]
        similarity = np.argsort(similarity)[::-1]
        #similarity = np.dot(self.M[:,m_id], self.M) / np.sqrt((self.M * self.M).sum(axis=0)) / np.sqrt((self.M[:,m_id] ** 2).sum())
        for i in similarity[:10]:
            print(movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
        return

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

Делаем все по презентации, добавляя Баеса

In [20]:
class SVD(Base):
    def __init__(self, matrix, ratings, k, amount_of_users, amount_of_movies, start):
        super().__init__(matrix, ratings, k, amount_of_users, amount_of_movies, start)
    def fit(self, ops, learning_rate = 0.01, regular = 0.001):
        inds = [i for i in range(len(self.ratings))]
        np.random.shuffle(inds)        
        for i in range(ops):
            ind = inds[i % len(self.ratings)]
            u_id = self.ratings.loc[ind]['user_id']
            m_id = self.ratings.loc[ind]['movie_id']
            rat = self.ratings.loc[ind]['rating']
            error = self.U[u_id] @ self.M[:,m_id] - rat #+ self.B_u[u_id] + self.B_m[:,m_id] + self.mu
            self.U[u_id], self.M[:,m_id] = self.U[u_id] - learning_rate * (error * (self.M[:,m_id]).T + regular * self.U[u_id]), self.M[:,m_id] - learning_rate * (error * (self.U[u_id]).T + regular * self.M[:,m_id]) 
            #self.B_u[u_id] = self.B_u[u_id] - learning_rate * (error + regular * self.B_u[u_id])
            #self.B_m[:,m_id] = self.B_m[:,m_id] - learning_rate * (error + regular * self.B_m[:, m_id])
            #self.mu -= learning_rate * error
            if (i % 100000 == 99999):
                print(self.rmse())

In [21]:
model = SVD(user_item_exp, ratings, 64, max(np.unique(ratings['user_id'])) + 1, max(np.unique(ratings['movie_id'])) + 1, np.sqrt(5))
model.fit(50000000)

1.4179336403245346
1.0826416840449478
0.9767049625648012
0.9266625624962188
0.8968629515676552
0.8802534058685242
0.8679174748681324
0.8585981805776317
0.8515954937348734
0.8451935544039382
0.8421826087761396
0.8382493717958327
0.834755198983399
0.8325736444645291
0.8280625078677111
0.8267478045267093
0.8238135086376697
0.8204708824832811
0.817778965720124
0.8140809787805837
0.812197209327838
0.8086219432528677
0.8048198288034484
0.8021484242946485
0.7966805512663256
0.793935738628228
0.7893788907621084
0.7841072555735213
0.7793526199050635
0.7734752045870481
0.7697989297678024
0.7638172277619504
0.7580549634291401
0.7532518906854005
0.7463418131395286
0.7414577701084257
0.7361796019673915
0.7295845734837918
0.7238247060479683
0.7173440890281692
0.7130868905641812
0.7063513309829934
0.7003158663311325
0.6953808355926823
0.6882875308901187
0.6830403879156141
0.6777766017524983
0.6710096507761328
0.6653758715780038
0.6590772984140165
0.6547052184140981
0.6482489470317624
0.64237120837445

0.27530610145567014
0.27448378747657126
0.2754982222609808
0.27490308596281793
0.27457409906666413
0.27484713665405675
0.27458502972821097
0.2751637205601568
0.27428089061560207
0.274117145619832
0.2739814780962666
0.2733375436640005
0.2742728739042617
0.2736853090789028
0.2734479786190569
0.27374486058807934
0.27340669151287544
0.27397980431906577
0.27311595340521744
0.27285773786298495
0.2727869830568699
0.27225063039783176
0.2730680671236802
0.27254904020554066
0.2723363350440423
0.27268466754590776
0.27228137462321667
0.27286081276315965
0.2720145001762435
0.271677766275433
0.2716072891576212
0.2711775851962021
0.271990186644265
0.27148031210963514
0.2712206970653837
0.27164744836776944
0.2711871053239348
0.27171276746115375
0.27093732307285745
0.270564327116661
0.2705021628378133
0.2701739082105919
0.27094436754721446
0.2704314042511989
0.27029387332484195
0.27057363537885504
0.2701118015295759
0.2707103052507983
0.2699924772088211
0.269492598257
0.26946362227502735
0.269218614145

Пора получать рекомендации и находить похожее

In [30]:
model.make_recomendation(4)

User films

3399    Hustler, The (1961)
1192    Star Wars: Episode VI - Return of the Jedi (1983)
2882    Fistful of Dollars, A (1964)
1196    Alien (1979)
1023    Die Hard (1988)
257    Star Wars: Episode IV - A New Hope (1977)
1959    Saving Private Ryan (1998)
476    Jurassic Park (1993)
1178    Star Wars: Episode V - The Empire Strikes Back...
1180    Raiders of the Lost Ark (1981)

Recomendation

['Goodbye Girl, The (1977)' 'Comedy|Romance']
['American Movie (1999)' 'Documentary']
['To Kill a Mockingbird (1962)' 'Drama']
['Fitzcarraldo (1982)' 'Adventure|Drama']
['Fargo (1996)' 'Crime|Drama|Thriller']
['Annie Hall (1977)' 'Comedy|Romance']
['Killer, The (Die xue shuang xiong) (1989)' 'Action|Thriller']
['Hoop Dreams (1994)' 'Documentary']
['Sling Blade (1996)' 'Drama|Thriller']
['Winslow Boy, The (1998)' 'Drama']


In [28]:
model.find_similar1(1)

Film                name                     category
0  Toy Story (1995)  Animation|Children's|Comedy
[['Toy Story (1995)' "Animation|Children's|Comedy"]]
[['Toy Story 2 (1999)' "Animation|Children's|Comedy"]]
[["Bug's Life, A (1998)" "Animation|Children's|Comedy"]]
[['Aladdin (1992)' "Animation|Children's|Comedy|Musical"]]
[['Battle of the Sexes, The (1959)' 'Comedy']]
[['Scorta, La (1993)' 'Thriller']]
[['Rear Window (1954)' 'Mystery|Thriller']]
[['Cabinet of Dr. Ramirez, The (1991)' 'Comedy']]
[["Pharaoh's Army (1995)" 'War']]
[['Of Love and Shadows (1994)' 'Drama']]


Почти все мультики)

In [29]:
model.find_similar2(10)

Film                name                   category
9  GoldenEye (1995)  Action|Adventure|Thriller
[['GoldenEye (1995)' 'Action|Adventure|Thriller']]
[['Tomorrow Never Dies (1997)' 'Action|Romance|Thriller']]
[['World Is Not Enough, The (1999)' 'Action|Thriller']]
[['For Your Eyes Only (1981)' 'Action']]
[['From Russia with Love (1963)' 'Action']]
[['Dr. No (1962)' 'Action']]
[['Live and Let Die (1973)' 'Action']]
[['Indiana Jones and the Last Crusade (1989)' 'Action|Adventure']]
[['Project Moon Base (1953)' 'Sci-Fi']]
[['Spy Who Loved Me, The (1977)' 'Action']]


Мега крутые похожие фильмы (несколько бондиан и индиана джонс)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

Используем статью http://yifanhu.net/PUB/cf.pdf

In [275]:
class ALS(Base):
    def __init__(self, matrix, ratings, k, amount_of_users, amount_of_movies, start):
        super().__init__(matrix, ratings, k, amount_of_users, amount_of_movies, start)
        self.M = self.M.T
        self.C = self.matrix.toarray() * 0.01
        self.C = self.C + 1
    def fit(self, ops):
        for i in range(ops):
            if (i % 2 == 0):
                self.M = model.step(self.U, self.C.T, self.matrix.T)
            else:
                self.U = model.step(self.M, self.C, self.matrix)
            print(self.my_rmse())
    def step(self, matr, C_to_use, rate, learning_rate = 0.01):
        ans = np.zeros((len(C_to_use), self.k))
        for i in range(len(C_to_use)):
            bracket = np.linalg.inv(matr.T @ np.diag(C_to_use[i]) @ matr + learning_rate * np.eye(matr.shape[1]))
            #print(bracket.shape)
            bracket = bracket @ matr.T @ np.diag(C_to_use[i])
            ans[i] = (bracket @ rate[i].T).T
        return ans
        
        
    def my_rmse(self):
        ans = ratings
        matr2 = ((self.U @ self.M.T))[ans["user_id"], ans["movie_id"]]
        matr1 = self.matrix[ans["user_id"], ans["movie_id"]]
        matr1 = np.squeeze(np.asarray(matr1))
        #print(matr1)
        #print(matr2)
        #print(np.squeeze(self.matrix[ans["user_id"], ans["movie_id"]]))
        return metrics.mean_squared_error(matr1, matr2, squared=True)

In [283]:
model = ALS(user_item_csr, implicit_ratings, 64, max(np.unique(ratings['user_id'])) + 1, max(np.unique(ratings['movie_id'])) + 1, 1)
model.fit(20)

0.455701184373924
0.3189876222076632
0.276442639871987
0.26368995632453907
0.2593893417743478
0.2567223388694169
0.2555222963371137
0.25455406588747614
0.25406650825405225
0.25360578433482406
0.25337037458614653
0.253114626146948
0.252989130716757
0.2528317016296451
0.2527591642380387
0.25265454001304594
0.2526091785070115
0.25253542346448976
0.2525049139341488
0.2524504411880954


Пора делать рекомендации (транспонируем так как в Базе другая размерность)

In [284]:
model.M = model.M.T
model.make_recomendation(4)
model.M = model.M.T

User films

3399    Hustler, The (1961)
2882    Fistful of Dollars, A (1964)
1196    Alien (1979)
1023    Die Hard (1988)
257    Star Wars: Episode IV - A New Hope (1977)
1959    Saving Private Ryan (1998)
476    Jurassic Park (1993)
1180    Raiders of the Lost Ark (1981)
1885    Rocky (1976)
1081    E.T. the Extra-Terrestrial (1982)

Recomendation

['Raiders of the Lost Ark (1981)' 'Action|Adventure']
['Star Wars: Episode IV - A New Hope (1977)'
 'Action|Adventure|Fantasy|Sci-Fi']
['Jaws (1975)' 'Action|Horror']
['E.T. the Extra-Terrestrial (1982)' "Children's|Drama|Fantasy|Sci-Fi"]
['Saving Private Ryan (1998)' 'Action|Drama|War']
['Terminator, The (1984)' 'Action|Sci-Fi|Thriller']
['Alien (1979)' 'Action|Horror|Sci-Fi|Thriller']
['Die Hard (1988)' 'Action|Thriller']
['Terminator 2: Judgment Day (1991)' 'Action|Sci-Fi|Thriller']
['Rocky (1976)' 'Action|Drama']


In [285]:
model.M = model.M.T
model.find_similar2(1)
model.M = model.M.T

Film                name                     category
0  Toy Story (1995)  Animation|Children's|Comedy
[['Toy Story (1995)' "Animation|Children's|Comedy"]]
[['Toy Story 2 (1999)' "Animation|Children's|Comedy"]]
[['Babe (1995)' "Children's|Comedy|Drama"]]
[["Bug's Life, A (1998)" "Animation|Children's|Comedy"]]
[['Aladdin (1992)' "Animation|Children's|Comedy|Musical"]]
[['Babe: Pig in the City (1998)' "Children's|Comedy"]]
[['Pleasantville (1998)' 'Comedy']]
[['Tarzan (1999)' "Animation|Children's"]]
[['Lion King, The (1994)' "Animation|Children's|Musical"]]
[['Hercules (1997)' "Adventure|Animation|Children's|Comedy|Musical"]]


In [286]:
model.M = model.M.T
model.find_similar2(10)
model.M = model.M.T

Film                name                   category
9  GoldenEye (1995)  Action|Adventure|Thriller
[['GoldenEye (1995)' 'Action|Adventure|Thriller']]
[['Tomorrow Never Dies (1997)' 'Action|Romance|Thriller']]
[['World Is Not Enough, The (1999)' 'Action|Thriller']]
[['Rush Hour (1998)' 'Action|Thriller']]
[['Die Hard: With a Vengeance (1995)' 'Action|Thriller']]
[['Licence to Kill (1989)' 'Action']]
[['View to a Kill, A (1985)' 'Action']]
[["Jackie Chan's First Strike (1996)" 'Action']]
[['Man with the Golden Gun, The (1974)' 'Action']]
[['For Your Eyes Only (1981)' 'Action']]


Получилось!!!

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

TODO https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных